In [1]:
from pathlib import Path

CURRENT_PATH = Path('.').absolute()
DB_PATH = CURRENT_PATH / 'db.sqlite3'

In [2]:
from providers.assrt.page_parser import AssrtDetailPage
from providers.assrt.search_parser import AssrtSearchPage

In [3]:
from SagasuSubs.database import SeriesCrud
from SagasuSubs.database import EpisodeCrud

series_crud = SeriesCrud(DB_PATH)
episode_crud = EpisodeCrud(DB_PATH)

In [ ]:
try:
    series_crud.create_table()
    episode_crud.create_table()
except Exception as e:
    print(e)

In [ ]:
from SagasuSubs.api.utils import NotebookAPIUtils
from SagasuSubs.api.auth import AuthTokenManager
from SagasuSubs import __file__ as sagasu_root

auth_file = Path(sagasu_root).absolute().parent.parent / 'auth.txt'

AuthTokenManager.load_file(auth_file)
api_utils = NotebookAPIUtils('http://127.0.0.1:3000')

In [ ]:
from tqdm.notebook import tqdm

main_progress = tqdm()

for page in api_utils.iterate_series():
    main_progress.update()
    main_progress.set_description(f'{page.meta.currentPage}/{page.meta.totalPages}')
    for series in page.items:
        series_dto,episode_dtos = series.to_dto()
        if series_crud.read(series_dto.id):
            continue
        series_crud.create(series_dto)
        for episode_dto in episode_dtos:
            if episode_crud.read(episode_dto.id):
                continue
            episode_crud.create(episode_dto)

In [5]:
from tqdm.notebook import tqdm

main_progress = tqdm(total=series_crud.count())

headers = {
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:92.0) Gecko/20100101 Firefox/92.0',
}
detail = AssrtDetailPage(headers=headers)
search = AssrtSearchPage(headers=headers)

for series in series_crud.iterate():
    resp = search.spider(series.name, 1)
    for index,result in enumerate(search.parse_page(resp)):
        if index >= 5:
            continue
        title,link = result
        main_progress.set_description(title)
        desc, down_link = detail.parse_page(detail.get_page(link))
        down_path = CURRENT_PATH / 'downloads' / series.name
        down_path.mkdir(parents=True, exist_ok=True)
        size,path = detail.download(down_link,down_path)
        with open(str(path)+'.txt','wt',encoding='utf-8') as f:
            f.write(desc)
        (down_path / 'data.json').write_text(series.json())
    main_progress.update()


  0%|          | 0/451 [00:00<?, ?it/s]

KeyboardInterrupt: 